In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd drive/Shareddrives/Filmes Pra TI - Machine Learning/Clone do Projeto - Luana/machinelearning

/content/drive/Shareddrives/Filmes Pra TI - Machine Learning/Clone do Projeto - Luana/machinelearning


In [4]:
ls

data/    Makefile    README.md    requirements.txt  test_environment.py
docs/    models/     references/  setup.py          tox.ini
LICENSE  notebooks/  reports/     src/


# Loading reduced train data

In [5]:
folder = 'data/processed/'

In [6]:
train = pd.read_csv(folder +'reduced_train_rating.csv')
val = pd.read_csv(folder +'reduced_val_rating.csv')
test = pd.read_csv(folder +'reduced_test_rating.csv')

Ensuring that all data splits have all selected movies and userids in order to create a map for new ids further

In [7]:
train.movieId.unique().shape, val.movieId.unique().shape, test.movieId.unique().shape

((2000,), (2000,), (2000,))

In [8]:
train.userId.unique().shape, val.userId.unique().shape, test.userId.unique().shape

((10000,), (10000,), (10000,))

In [9]:
train.head()

,userId,movieId,rating,timestamp,Liked,timestamp2
0,11,4643,4.5,2009-01-01 04:02:09,1,1230782529
1,11,170,5.0,2009-01-01 04:02:14,1,1230782534
2,11,531,4.5,2009-01-01 04:02:19,1,1230782539
3,11,616,4.0,2009-01-01 04:02:22,1,1230782542
4,11,1093,3.0,2009-01-01 04:02:41,1,1230782561


## Preprocess Data

In [10]:
train.movieId.min(), train.movieId.max(), train.movieId.unique().shape[0]

(1, 109374, 2000)

In [11]:
train.userId.min(), train.userId.max(), train.userId.unique().shape[0]

(11, 138474, 10000)

We can see that the ids go from 1 to 109374 but there are 2000 movies only. Same thing for users Id.
So we need to remap the movieId to new indexed from [0,2000] and userId to [0, 10000]

In [12]:
movie_ids_dict = {}
user_ids_dict = {}

sorted_moviesId = train.movieId.unique()
sorted_moviesId.sort()

sorted_usersId = train.userId.unique()
sorted_usersId.sort()

M = train.movieId.unique().shape[0]
N = train.userId.unique().shape[0]

for i in range(M):
    movieId = sorted_moviesId[i]
    movie_ids_dict[movieId] = i

for i in range(N):
    userId = sorted_usersId[i]
    user_ids_dict[userId] = i

In [13]:
def map_ids(df):
    df['movieIndex'] = df.apply(lambda row: movie_ids_dict[row['movieId']], axis=1)
    df['userIndex'] = df.apply(lambda row: user_ids_dict[row['userId']], axis=1)
    return df

In [14]:
train = map_ids(train)
test = map_ids(test)
val = map_ids(val)

In [15]:
train.head()

,userId,movieId,rating,timestamp,Liked,timestamp2,movieIndex,userIndex
0,11,4643,4.5,2009-01-01 04:02:09,1,1230782529,1495,0
1,11,170,5.0,2009-01-01 04:02:14,1,1230782534,98,0
2,11,531,4.5,2009-01-01 04:02:19,1,1230782539,276,0
3,11,616,4.0,2009-01-01 04:02:22,1,1230782542,316,0
4,11,1093,3.0,2009-01-01 04:02:41,1,1230782561,475,0


In [16]:
val.drop(columns=['timestamp', 'timestamp2','userId','movieId'], inplace=True)
test.drop(columns=['timestamp', 'timestamp2','userId','movieId'], inplace=True)
train.drop(columns=['timestamp', 'timestamp2','userId','movieId'], inplace=True)

In [17]:
train.to_csv(folder + 'very_reduced_train_ratings.csv', index=False)
test.to_csv(folder + 'very_reduced_test_ratings.csv', index=False)
val.to_csv(folder + 'very_reduced_val_ratings.csv', index=False)

In [18]:
import pickle

with open(folder +'IDS_DICT/movies_ids.pickle', 'wb') as handle:
    pickle.dump(movie_ids_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(folder +'IDS_DICT/users_ids.pickle', 'wb') as handle:
    pickle.dump(user_ids_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open(folder +'IDS_DICT/movies_ids.pickle', 'rb') as handle:
#     b = pickle.load(handle)